In [ ]:
# In this micro-project, we'll practice using Jupyter Notebook on the server to design a CNN for
# MNIST image classification and train it using the GPUs.

# Imports various libraries (note that we are using tf.Keras, not vanilla Keras).
from tensorflow import keras
from tensorflow.keras.datasets import mnist
import tensorflow.keras.backend as K
import matplotlib.pyplot as plt

In [ ]:
# Loads the MNIST dataset.
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [ ]:
# Plots an MNIST digit.
plt.title("Label is {}".format(y_train[0]))
plt.imshow(x_train[0], cmap="gray")

In [ ]:
# Preprocesses images.
print(x_train.shape, x_test.shape)

# Adds an extra dimension to the image tensor.
# This represents the "channels" of the image.
# BW images have one channel; RGB images have three.
x_train = x_train.reshape(60000, 28, 28, 1)
x_test = x_test.reshape(10000, 28, 28, 1)

# Converts the tensors from uint8 to float32.
# Most CNNs expect inputs of type float32.
x_train = x_train.astype("float32")
x_test = x_test.astype("float32")

# Scales values to be between 0 and 1.
x_train /= 255
x_test /= 255

print(x_train.shape, x_test.shape)

In [ ]:
# Preprocesses labels.
print(y_train.shape, y_test.shape)

# Converts labels from numbers (e.g., 7)
# to one-hot vectors (e.g., [0, 0, 0, 0, 0, 0, 0, 1, 0, 0]).
y_train = keras.utils.to_categorical(y_train, num_classes=10)
y_test = keras.utils.to_categorical(y_test, num_classes=10)

print(y_train.shape, y_test.shape)

In [ ]:
# Defines a simple CNN with two convolutional layers and two dense layers.
# Note that this CNN uses the tf.Keras Functional API.
def cnn():
    inputs = keras.layers.Input(shape=x_train.shape[1:])
    conv1 = keras.layers.Conv2D(32, 3, activation="relu")(inputs)
    conv2 = keras.layers.Conv2D(64, 3, activation="relu")(conv1)
    pool = keras.layers.MaxPool2D()(conv2)
    drop1 = keras.layers.SpatialDropout2D(0.5)(pool)
    flat = keras.layers.Flatten()(drop1)
    dense = keras.layers.Dense(128, activation="relu")(flat)
    drop2 = keras.layers.Dropout(0.5)(dense)
    softmax = keras.layers.Dense(10, activation="softmax")(drop2)

    model = keras.Model(inputs=inputs, outputs=softmax)
    model.summary()
    return model

In [ ]:
# Instantiates a CNN and compiles it with the Adam optimizer, categorical_crossentropy loss, and accuracy metric.
model = cnn()
model.compile(optimizer="Adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Fits the model for 20 epochs using the GPUs.
model.fit(x_train, y_train, batch_size=128, epochs=20, validation_data=(x_test, y_test))

In [ ]:
# Evaluates the model on the test dataset.
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])